**This notebook represents a conceptual workflow to extract and visualize planned (GTFS) and actual (Siri) bus data of a single bus trip**.

A single bus trip details is identified by an initial user's input of **date and line number**, which are used in GTFS queries, that returns to the user data about possible **agency, starting time and starting stop** from which one option should be picked to uniqely identify the single trip.

The script requires that the data for the required date will already be saved in the user "DATA_DIR" directory:
* GTFS file (can be obtained by running gtfs_stats.py with the relevant date set at gtfs_stats_conf.py)
* Route stats and trip stats data frames (can be obtained by running gtfs_stats.py on the GTFS mentioned above)
* Raw Siri log files (As for now should be manually downloaded from our google drive storage)

(GTFS stats and Siri data are supposed to be avilable online soon.)

# Imports and Utils

In [ ]:
import pandas as pd
import os.path
import gzip
import glob
import numpy as np
import partridge as ptg
import datetime

In [ ]:
# DATA_DIR = 'PATH/TO/DATA/DIR' ## Your input here
DATA_DIR = r'C:\Users\adiwa\Google Drive\OpenBusAdi\data' ## Your input here

In [ ]:
def merc_row(row, lat_col='lat', lon_col='lon'):
    """ Convert GPS coordinates to x,y format"""
    lat = float(row[lat_col])
    lon = float(row[lon_col])
    
    new_row = row.copy()
    
    if lon == 0:
        new_row['x'] = np.nan
        new_row['y'] = np.nan
        return new_row
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    new_row['x'] = x
    new_row['y'] = y
    return new_row

# GTFS data

## Date + Line number --> agency

In [ ]:
date = '2018-10-11' ## Your input here

In [ ]:
# Get the relevant GTFS stats tables:
rstats = pd.read_pickle(os.path.join(DATA_DIR, 'gtfs_stats_MOD_ffill',
                                     '{}_route_stats.pkl.gz'.format(date)))

In [ ]:
line_number = '108' ## Your input here

In [ ]:
agency_options =  rstats.loc[rstats.route_short_name == line_number, 
                             ['agency_id', 'agency_name']].drop_duplicates()
agency_options = agency_options.sort_values('agency_name')
print('We found {} providers for this line, please choose agency_id:'.format(agency_options.shape[0]))
print('\n'.join(['----'.join(x) for _, x in agency_options.iterrows()]))

## Agency --> Hours

In [ ]:
agency_id = '18'  ## Your input here

In [ ]:
tstats = pd.read_pickle(os.path.join(DATA_DIR, 'gtfs_stats_MOD_ffill',
                                     '{}_trip_stats.pkl.gz'.format(date)))

In [ ]:
# start_time
optional_start_time = tstats.loc[(tstats.route_short_name == line_number) & 
                                 (tstats.agency_id == agency_id)].start_time.unique()
optional_start_time = sorted(optional_start_time)
print('We found {} optional start times, please choose one:'.format(len(optional_start_time)))
print('\n'.join(optional_start_time))

## Time --> start and end stops

In [ ]:
start_time = '07:40:00'  ## Your input here

In [ ]:
optional_starts = tstats.loc[(tstats.route_short_name == line_number) & 
                             (tstats.agency_id == agency_id) &
                             (tstats.start_time == start_time)]
if optional_starts.empty:
    print('Bad start time')
elif optional_starts.shape[0] == 1:
    route_id = optional_starts.route_id.values[0]
    start_id, end_id = optional_starts[['start_stop_id', 'end_stop_id']].values[0]
    print(optional_starts[['start_stop_id', 'end_stop_id']])
else:
    optional_starts_str = optional_starts.apply(lambda r: 'From {} {} To {} {}'.format(r.start_stop_name,
                                                                                       r.start_stop_id,
                                                                                       r.end_stop_name,
                                                                                       r.end_stop_id),
                                                axis=1)

    print('We found {} optional directions, please choose one:'.format(optional_starts.shape[0]))
    print('\n'.join(optional_starts_str.values))

## From start+end+time --> route id --> shape_id

In [ ]:
route_id, trip_id, shape_id = optional_starts.loc[(optional_starts.start_stop_id == start_id) &
                                                  (optional_starts.end_stop_id == end_id), 
                                                  ['route_id', 'trip_id', 'shape_id']].values[0]

route_id, trip_id, shape_id

## Finally.... get the route shape and stops

In [ ]:
gtfs_path = os.path.join(DATA_DIR, 'gtfs_feeds', '{}.zip'.format(date))
feed = ptg.feed(gtfs_path, view={
    'trips.txt': {'trip_id': [trip_id]},
})

In [ ]:
trip_shape = feed.shapes[feed.shapes.shape_id == shape_id]
trip_shape = trip_shape.apply(merc_row, lat_col='shape_pt_lat', lon_col='shape_pt_lon', axis=1)
trip_shape = trip_shape.dropna(how='any', subset=['x'])
trip_shape = trip_shape.set_index('shape_pt_sequence')[['x', 'y']].sort_index()
trip_shape.shape

In [ ]:
# stops of this trip with times:
stop_times = feed.stop_times # should have only the relevant trip id!
stop_times = stop_times[stop_times.trip_id == trip_id]
time_cols = ['arrival_time', 'departure_time']
stop_times[time_cols] = stop_times[time_cols].apply(pd.to_timedelta, unit='s')

# add stops details (geo, name)
stop_times = stop_times.merge(feed.stops, on='stop_id')
stop_times = stop_times.apply(merc_row, lat_col='stop_lat', lon_col='stop_lon', axis=1)

# Siri data

In [ ]:
def parse_trips(path):
    trip = []
    with gzip.open(path, 'rb') as f:
        for line in f:
            line = line.strip()
            if line:
                line = line.split(b',')[2:]
                line = [element.decode("utf-8") for element in line]
                trip.append({"agency": line[0], "route_id": line[1], "line_num": line[2]
                                , "service_id": line[3], "start_time": line[4], "bus_id": line[5],
                             "end_time": line[6], "time_recorded": line[7],
                             "coordinates": (line[8], line[9])})
    return trip

In [ ]:
trips = None
dir_path = r'C:\Users\adiwa\Google Drive\OpenBusAdi\data\siri_logs\Siri-{}'.format(date)
if not os.path.exists(dir_path):
    dir_path = r'C:\Users\adiwa\Google Drive\OpenBusAdi\data\siri_logs\Siri-{}'.format(date[:7])
for path in glob.glob(os.path.join(dir_path, 'siri_rt_data.{}.*.log.gz'.format(date))): 
    print(path)
    cur_trips = pd.DataFrame(parse_trips(path))
    trips = pd.concat([trips, cur_trips], axis=0)

In [ ]:
## TODO - this code is very specific to the string format of the time - will it stay like this?
siri_data = trips[(trips.agency == agency_id) & (trips.route_id == route_id) & 
                  (trips.start_time == ':'.join(start_time.split(':')[:-1]))]
siri_data = siri_data.drop_duplicates()
print(siri_data.shape)
siri_data = siri_data.sort_values('time_recorded')

siri_data = pd.concat([siri_data, pd.DataFrame(siri_data.coordinates.values.tolist(), 
                                               index=siri_data.index, columns=['lat', 'lon'])], 
                      axis=1, sort=False)
siri_data.shape

In [ ]:
siri_data = siri_data.apply(merc_row, axis=1)
siri_data = siri_data.dropna(how='any', subset=['x'])
siri_data = siri_data.drop_duplicates(['x', 'y', 'time_recorded']).sort_values('time_recorded')

siri_data['serial_number'] = siri_data.reset_index(drop=True).index.values + 1
siri_data.shape

# Bokeh visualization

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

## Use this import to save html file instead of previewing it inside the notebook
# from bokeh.io import output_file, save 

from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.palettes import inferno, magma
from bokeh.models import ColumnDataSource, LabelSet, HoverTool, Circle, Square

In [ ]:
# Plot siri:
plot_width  = int(540)
plot_height = plot_width

# zoom and scale
p = figure(tools='pan,wheel_zoom,box_zoom,reset,save',
           active_drag='pan', active_scroll='wheel_zoom', toolbar_location='left',
           x_axis_type="mercator", y_axis_type="mercator",
           title='GPS data for line {} at {}, {}'.format(line_number, date, start_time))
# map
p.add_tile(CARTODBPOSITRON)
# GPS points
siri_source = ColumnDataSource(siri_data)
siri_points = Circle(x='x', y='y', size=10, fill_color='blue')
siri_gl = p.add_glyph(source_or_glyph=siri_source, glyph=siri_points)

# Points hover tool
siri_tooltips = [
    ("serial_number", "@serial_number"),
    ("time_recorded", "@time_recorded"),
    ]
siri_hover = HoverTool(renderers=[siri_gl],
                       tooltips=siri_tooltips)
p.add_tools(siri_hover)


# GPS annotations
labels = LabelSet(x='x', y='y', text='serial_number', level='glyph',
                  x_offset=0.1, y_offset=0.2, source=siri_source, render_mode='canvas')
p.add_layout(labels)

In [ ]:
# Plot GTFS:
gtfs_source = ColumnDataSource(trip_shape)
p.line(x='x', y='y', source=gtfs_source, line_width=2, color='yellow')

In [ ]:
stops_source = ColumnDataSource(stop_times)
stops_squares = Square(x='x', y='y', size=10, fill_color='yellow')
stops_gl = p.add_glyph(source_or_glyph=stops_source, glyph=stops_squares)

# Squares hover tool
stops_tooltips = [
    ("arrival_time", "@arrival_time{%T}"),
    ("stop_name", "@stop_name"),
    ]
stops_hover = HoverTool(renderers=[stops_gl],
                        tooltips=stops_tooltips, formatters={'arrival_time': 'datetime'})
p.add_tools(stops_hover)

In [ ]:
show(p)

In [ ]:
# output_file('GPS_data_line_{}_{}_{}.html'.format(line_number, date, start_time.replace(':', '')),
#             mode='inline') 
# save(p) 